# Wczytywanie danych z SQL

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('database.sqlite')

# Sprawdźmy, jakie tabele są dostępne w bazie danych
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
available_tables = cursor.fetchall()

print("Dostępne tabele w bazie danych:")
for table in available_tables:
    print(f"- {table[0]}")

sales_df = pd.read_sql("SELECT * FROM Sales", conn)
customers_df = pd.read_sql("SELECT * FROM Customers", conn)
cities_df = pd.read_sql("SELECT * FROM Cities", conn)
item_master_df = pd.read_sql("SELECT * FROM Item_Master", conn)
sales_rep_df = pd.read_sql("SELECT * FROM Sales_Rep", conn)
kursy = pd.read_sql("SELECT * FROM kursy_walut", conn)



Dostępne tabele w bazie danych:
- Cities
- Customers
- Item_Master
- Sales_Rep
- Sales
- kursy_walut


# Łączenie tabel

In [2]:
# 1. Łączenie Sales z Customers
sales_data = pd.merge(sales_df, customers_df, on='Customer Number', how='left')

# 2. Dołączanie Cities
print("Dołączanie Cities...")
sales_data = pd.merge(sales_data, cities_df, on='City Code', how='left')

# 3. Dołączanie Item_Master
print("Dołączanie Item_Master...")
sales_data = pd.merge(sales_data, item_master_df, on='Item Number', how='left')

print("Utworzono sales_data.")
print(f"Rozmiar końcowej tabeli: {sales_data.shape}")
print("Podgląd połączonych danych:")
print(sales_data.head())

Dołączanie Cities...
Dołączanie Item_Master...
Utworzono sales_data.
Rozmiar końcowej tabeli: (457837, 27)
Podgląd połączonych danych:
   %KEY    Cost  Customer Number                 Date  GrossSales  \
0  3428 -513.15         10012226  2019-01-12 00:00:00   -573.3835   
1  3429 -105.93         10012226  2019-01-12 00:00:00   -204.6638   
2  3430  -88.07         10012226  2019-01-12 00:00:00   -165.8016   
3  3431  -43.12         10012226  2019-01-12 00:00:00   -118.3703   
4  3432  -37.98         10012226  2019-01-12 00:00:00   -102.3319   

          Invoice Date  Invoice Number                   Item Desc  \
0  2019-01-12 00:00:00          318960     Cutting Edge Sliced Ham   
1  2019-01-12 00:00:00          318960  Washington Cranberry Juice   
2  2019-01-12 00:00:00          318960             Moms Sliced Ham   
3  2019-01-12 00:00:00          318960                 Tip Top Lox   
4  2019-01-12 00:00:00          318960        Just Right Beef Soup   

   Item Number  Margin  ...  

#Agregacja sprzedaży

In [3]:
# łączna sprzedaż dla każdego regionu
# rupujemy po region ( doszedł z tabeli Cities) i sumujemy kolumnę sales
region_sales = sales_data.groupby('Region')['Sales'].sum().sort_values(ascending=False)


print("\nŁączna sprzedaż w regionach:")

print(region_sales)


Łączna sprzedaż w regionach:
Region
USA        2.132156e+08
UK         1.311867e+08
Japan      5.641052e+07
Nordic     5.145371e+07
Spain      1.613757e+07
Germany    1.592449e+07
Name: Sales, dtype: float64


# Filtrowanie danych

In [4]:
query_task_4 = """
SELECT *
FROM Sales s
LEFT JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"  -- Twoje łączenie
WHERE
    s.Sales > 1000
    AND s.Margin < 0
ORDER BY
    s.Margin ASC
LIMIT 10;
"""
top_10_sql = pd.read_sql(query_task_4, conn)

print("\n--- ZADANIE 4: Top 10 transakcji (Sales > 1000, Margin < 0) ---")
columns_to_show = ['Sales', 'Margin', 'Sales Rep ID']
print(top_10_sql[columns_to_show])


--- ZADANIE 4: Top 10 transakcji (Sales > 1000, Margin < 0) ---
     Sales   Margin  Sales Rep ID
0  2177.28 -3764.40           141
1  2177.28 -3764.40           141
2  2177.28 -3764.40           141
3  2177.28 -3764.40           141
4  2268.00 -3673.68           141
5  2268.00 -3673.68           141
6  2268.00 -3673.68           141
7  2268.00 -3673.68           141
8  2268.00 -3673.68           141
9  2268.00 -3673.68           141


# Grupy produktowe

In [5]:
# używamy ramki danych z kroku 2 (sales data)
product_group_stats = sales_data.groupby('Product Group')[['Sales', 'Sales Qty']].sum().reset_index()

# sortowanie i wybór Top 5
top_5_product_groups = product_group_stats.sort_values(by='Sales', ascending=False).head(5)
print("\n--- Top 5 grup produktowych z najwyższą sprzedażą ---")
print(top_5_product_groups)


--- Top 5 grup produktowych z najwyższą sprzedażą ---
      Product Group         Sales  Sales Qty
11          Produce  1.114055e+08  1951190.0
5   Canned Products  9.107153e+07  2672814.0
7              Deli  6.872424e+07   686456.0
9      Frozen Foods  4.464965e+07   425480.0
13           Snacks  3.975459e+07   436887.0


# Podział plików sprzedaży na waluty

In [6]:
import os

folder_name = 'dane_walutowe'
os.makedirs(folder_name, exist_ok=True)

lista_walut = sales_df['Currency'].unique()

print(f"Znalezione waluty do przetworzenia: {lista_walut}")

for waluta in lista_walut:

    dane_dla_waluty = sales_df[sales_df['Currency'] == waluta]

    nazwa = waluta.replace('/', '_')
    nazwa_pliku = f"waluta_{nazwa}.csv"

    # sklejamy folder z nazwą pliku -> wynik np.: "dane_walutowe/waluta_EUR.csv"
    sciezka_zapisu = os.path.join(folder_name, nazwa_pliku)

    # index=False sprawia, że nie zapisujemy numerów wierszy (0, 1, 2...)
    dane_dla_waluty.to_csv(sciezka_zapisu, index=False)

    print(f" -> Zapisano plik: {nazwa_pliku}")

print("Wszystkie pliki zostały utworzone.")

Znalezione waluty do przetworzenia: ['AUD' 'EUR' 'USD' 'GBP' 'CHF' 'CAD']
 -> Zapisano plik: waluta_AUD.csv
 -> Zapisano plik: waluta_EUR.csv
 -> Zapisano plik: waluta_USD.csv
 -> Zapisano plik: waluta_GBP.csv
 -> Zapisano plik: waluta_CHF.csv
 -> Zapisano plik: waluta_CAD.csv
Wszystkie pliki zostały utworzone.


# Transakcje sprzedawców


In [7]:
query_task_7 = """
SELECT
    sr."Sales Rep Name",
    COUNT(s."Order Number") as "Order Number"
FROM Sales s
LEFT JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"
GROUP BY
    sr."Sales Rep ID",
    sr."Sales Rep Name"
ORDER BY
    "Order Number" DESC
LIMIT 3;
"""
top_3_sellers_sql = pd.read_sql(query_task_7, conn)

print("\n--- Top 3 Sprzedawców (SQL) ---")
print(top_3_sellers_sql)


--- Top 3 Sprzedawców (SQL) ---
  Sales Rep Name  Order Number
0    Brad Taylor         37720
1      TAGnology         28466
2  Karl Anderson         21590


# Wartość sprzedaży w walutach

In [8]:
sales_with_cur = pd.merge(sales_data, kursy, left_on='Currency', right_on='code', how='left')
# przeliczenie sprzedaży na PLN
sales_with_cur['Sales_PLN'] = sales_with_cur['Sales'] * sales_with_cur['mid']

# łączna sprzedaż w PLN dla regionu i waluty
region_currency_stats = sales_with_cur.groupby(['Region', 'Currency'])['Sales_PLN'].sum().reset_index()

region_currency_stats = region_currency_stats.sort_values(by=['Region', 'Sales_PLN'], ascending=[True, False])

print("\n--- Łączna sprzedaż w PLN wg regionu i waluty ---")
print(region_currency_stats)


--- Łączna sprzedaż w PLN wg regionu i waluty ---
     Region Currency     Sales_PLN
4   Germany      GBP  1.423193e+07
5   Germany      USD  1.065586e+07
2   Germany      CHF  1.024049e+07
1   Germany      CAD  8.762595e+06
0   Germany      AUD  8.395630e+06
3   Germany      EUR  8.262100e+06
10    Japan      GBP  4.953245e+07
11    Japan      USD  3.736031e+07
8     Japan      CHF  3.338006e+07
6     Japan      AUD  3.128501e+07
9     Japan      EUR  3.113443e+07
7     Japan      CAD  3.066560e+07
16   Nordic      GBP  4.817655e+07
14   Nordic      CHF  3.371326e+07
17   Nordic      USD  3.256812e+07
13   Nordic      CAD  2.806473e+07
15   Nordic      EUR  2.706461e+07
12   Nordic      AUD  2.685829e+07
22    Spain      GBP  1.327573e+07
20    Spain      CHF  1.149454e+07
23    Spain      USD  9.810474e+06
19    Spain      CAD  9.740943e+06
21    Spain      EUR  8.784132e+06
18    Spain      AUD  8.043451e+06
28       UK      GBP  1.070120e+08
29       UK      USD  8.481764e+07
26  

# Analiza z wykorzystaniem apply

In [9]:

def klasyfikuj(margin):
    if margin < 0:
        return "Strata"
    elif margin == 0:
        return "Neutral"
    else:
        return "Zysk"

# sales_data['Margin'] to nasza kolumna z liczbami
# .apply(klasyfikuj) bierze każdą liczbę po kolei i przepuszcza przez funkcję
sales_data['Profit Category'] = sales_data['Margin'].apply(klasyfikuj)

print("\n--- Rozkład kategorii zysku ---")
print(sales_data['Profit Category'].value_counts())

print("\n--- Podgląd danych z nową kolumną ---")
print(sales_data[['Margin', 'Profit Category']].head(10))


--- Rozkład kategorii zysku ---
Profit Category
Zysk       438855
Strata      18844
Neutral       138
Name: count, dtype: int64

--- Podgląd danych z nową kolumną ---
   Margin Profit Category
0  -37.29          Strata
1  -90.54          Strata
2  -71.10          Strata
3  -70.52          Strata
4  -60.26          Strata
5  -32.79          Strata
6  -19.89          Strata
7  -52.12          Strata
8   -2.08          Strata
9    1.00            Zysk


# Tabela przestawna i eksport

In [10]:
# --- ZADANIE 10: Tabela przestawna i eksport ---

# 1. Tworzenie tabeli przestawnej
# values  -> co liczymy (Sprzedaż)
# index   -> co ma być w wierszach (Region)
# columns -> co ma być w kolumnach (Grupa produktowa)
# aggfunc -> funkcja, której używamy (suma)
pivot = sales_data.pivot_table(values='Sales',
                               index='Region',
                               columns='Product Group',
                               aggfunc='sum')

print("\n--- Tabela Przestawna (Region x Grupa Produktowa) ---")
print(pivot)

output_file = 'sprzedaz_regiony_produkty.xlsx'
pivot.to_excel(output_file)

print("Tabela zapisana do pliku: {output_file}")


--- Tabela Przestawna (Region x Grupa Produktowa) ---
Product Group  Alcoholic Beverages  Baked Goods  Baking Goods    Beverages  \
Region                                                                       
Germany                  175853.63    176290.48     625397.71   1034551.06   
Japan                   5724265.60    419288.71    2505992.79   3381942.34   
Nordic                   860300.03    357667.08    1077306.22   1467090.86   
Spain                    287017.36    116030.33     527514.45   1196922.17   
UK                       730977.04    486163.08   19702480.96  10243352.10   
USA                     2914969.93    534964.90    5994748.11  13170348.17   

Product Group  Breakfast Foods  Canned Products        Dairy         Deli  \
Region                                                                      
Germany               74436.72       2037546.56   1296268.35   2539111.57   
Japan                182716.49       6414767.11   6604613.56   9494626.98   
Nordic      

#Propozycje podsumowań

## Ranking Top 5 miast z największą sprzedażą

In [11]:
query_task_11 = """
SELECT
    c.City,
    c.Region,
    COUNT(s."Order Number") as "Liczba Zamówień",
    SUM(s.Sales) as "Łączna Sprzedaż"
FROM Sales s
JOIN Customers cust ON s."Customer Number" = cust."Customer Number"
JOIN Cities c ON cust."City Code" = c."City Code"
GROUP BY
    c.City, c.Region
ORDER BY
    "Łączna Sprzedaż" DESC
LIMIT 5;
"""

top_cities_summary = pd.read_sql(query_task_11, conn)

print("\n--- Top 5 miast z największą sprzadażą ---")
print(top_cities_summary)


--- Top 5 miast z największą sprzadażą ---
        City Region  Liczba Zamówień  Łączna Sprzedaż
0    Cardiff     UK             3606      35325747.33
1  Nashville    USA             4876      28464238.72
2  Leicester     UK             8506      25016681.60
3     Denver    USA             3922      23648064.07
4  Baltimore    USA              823      19808067.72


## Kto generuje największy przychód

In [12]:
query_task_11_2 = """
SELECT
    sr."Sales Rep Name",
    SUM(s.Sales) as "Łączny Przychód",
    AVG(s.Margin) as "Średnia Marża"
FROM Sales s
JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"
GROUP BY
    sr."Sales Rep Name"
ORDER BY
    "Łączny Przychód" DESC
LIMIT 5;
"""

top_revenue_reps = pd.read_sql(query_task_11_2, conn)

print("\n--- Najlepsi Sprzedawcy (wg Przychodu) ---")
pd.options.display.float_format = '{:,.2f}'.format
print(top_revenue_reps)


--- Najlepsi Sprzedawcy (wg Przychodu) ---
  Sales Rep Name  Łączny Przychód  Średnia Marża
0   Judy Thurman    64,531,648.58      11,117.41
1   Stewart Wind    58,043,432.70       4,421.05
2       Lee Chin    41,959,884.90      18,709.61
3      John Greg    20,597,023.77         667.66
4    Brad Taylor    18,825,697.39         212.76
